![](https://github.com/destination-earth/DestinE-DataLake-Lab/blob/main/img/DestinE-banner.jpg?raw=true)

**Copyright:** 2025 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Serena Avolio (EUMETSAT/Starion)

<div class="alert alert-block alert-success">
<h3>Destination Earth - Aviso notification for DT data availability</h3></div>

<div class="alert alert-block alert-warning">
<b> References: </b><ul>
    <li><a href="https://pyaviso.readthedocs.io/en/latest/">Pyaviso</a></li>
    <li><a href="https://github.com/destination-earth-digital-twins/aviso-examples/tree/main">Aviso examples</a></li>
    <li> <a href="https://destine.ecmwf.int/weather-induced-extremes-digital-twin-1/">Weather-Induced Extremes Digital Twin (Extremes DT)</a></li>
    <li> <a href="https://confluence.ecmwf.int/display/DDCZ/Extremes+DT+data+catalogue">Extreme DT data catalogue</a></li></ul>
<b> Credit: </b><ul>
    <li> The pyaviso package is provided by the European Centre for Medium-Range Weather Forecasts (ECMWF).</li></ul>
</div>


This notebook shows how to check the data availablility for the Weather-Induced Extremes Digital Twin (Extremes DT) using the ECMWF Aviso package.

<H2>Install the pyaviso package</H2>

In [1]:
pip  install  pyaviso --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


<H2>Import pyaviso</H2>
Import pyaviso and other useful libraries. Defining constants and functions.

In [2]:
from datetime import datetime
from pprint import pprint as pp

from pyaviso import NotificationManager, user_config

In [3]:
LISTENER_EVENT = "data"  # Event for the listener, options are mars and dissemination
TRIGGER_TYPE = "function"  # Type of trigger for the listener

<H2>Defining the data to be notified</H2>
The following request describes the data whose availability we want to be notified

In [4]:
REQUEST = {
    "class": "d1",
    "expver": "0001",
    "stream": "wave",
    "type": "fc",
    "time": "00",
    "step": "0",
    "levtype": "sfc",
#    "levelist": "",
#    "param": "168"   
}  # Request configuration for the listener


<H2>Aviso configuration</H2>

In [5]:
CONFIG = {
    "notification_engine": {
        "host": "aviso.lumi.apps.dte.destination-earth.eu",
        "port": 443,
        "https": True,
    },
    "configuration_engine": {
        "host": "aviso.lumi.apps.dte.destination-earth.eu",
        "port": 443,
        "https": True,
    },
    "schema_parser": "generic",
    "remote_schema": True,
    "auth_type": "none",
    "quiet" : True
}  # manually defined configuration


<H2>Searching for old notifications</H2>
Ssearching for old notifications where available. 
This way users can explicitly replay past notifications and executes triggers. 

In [6]:
import sys
START_DATE = datetime(2020, 12, 12)  # Start date for the notification listener

In [7]:
def triggered_function(notification):
    """
    Function for the listener to trigger.
    """
    
    #pp(notification)
    # Access the date field
    date_str = notification['request']['date']    

    # Convert the date string to a datetime object
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    formatted_date = date_obj.strftime('%Y-%m-%d')
    pp("ExtremeDT data available=>" + formatted_date)
    
    

In [8]:
def create_hist_listener():
    """
    Creates and returns a listener configuration.
    """

    trigger = {
        "type": TRIGGER_TYPE,
        "function": triggered_function,
    }  # Define the trigger for the listener
    
    # Return the complete listener configuration
    return {"event": LISTENER_EVENT, "request": REQUEST, "triggers": [trigger]}


In [ ]:
try:
    listener = create_hist_listener()  # Create listener configuration
    listeners_config = {"listeners": [listener]}  # Define listeners configuration
    config = user_config.UserConfig(**CONFIG)
    print("loaded config:")
    pp(CONFIG)
    nmh = NotificationManager()  # Initialize the NotificationManager

    nmh.listen(
        listeners=listeners_config, from_date=START_DATE, config=config
    )  # Start listening
except Exception as e:
    print(f"Failed to initialize the Notification Manager: {e}")

loaded config:
{'auth_type': 'none',
 'configuration_engine': {'host': 'aviso.lumi.apps.dte.destination-earth.eu',
                          'https': True,
                          'port': 443},
 'notification_engine': {'host': 'aviso.lumi.apps.dte.destination-earth.eu',
                         'https': True,
                         'port': 443},
 'quiet': True,
 'remote_schema': True,
 'schema_parser': 'generic'}
